In [1]:
import glob
input_files = glob.glob("../../data/ferg_geo_data/geotagged_combined_json/*")
print len(input_files)

90966


In [3]:
import numpy as np

x = np.array_split(input_files, 100)

In [4]:
len(x[0])

910

In [45]:
# get gender info
import io
from urllib2 import Request, urlopen, URLError
import pandas as pd 

gend_name = pd.DataFrame([x.split() for x in io.open("../data/ferg_geo_data/id_to_gender.txt")])
gend_name.columns = ['uid','gender']

In [3]:
# read census data 

fields_of_interest = ['B01003_001E','B01001_002E','B01001_026E','B02001_002E','B02001_003E','B01002_001E',
                      'B19001_001E','B19013_001E','B03003_002E','B03003_003E','B19083_001E',
                      'B19301_001E','B19301H_001E','B19301B_001E','B19301I_001E','B23025_002E','B23025_007E']

state_to_abbrev = {x.strip().split("\t")[1]:x.split("\t")[0] for x in 
                       open("../../identity_extraction/analysis/state_abbrev.tsv","r")}
state_to_abbrev['DC'] = 'District of Columbia'
state_to_abbrev['PR'] = 'Puerto Rico'

c = io.open("../data/ferg_geo_data/uid_to_census_block.txt")
counties = set()
full = set()
for line in c:
    uid, x = line.split()
    if uid not in gend_name:
        continue
    state, county, tract,block_group, block = x[:2], x[2:5],x[5:11],x[11],x[12:15]
    counties.add(state+"_"+county)
    full.add(x)

print len(counties), len(full)
c.close()

def get_loc_data(state, county):
    
    q = "http://api.census.gov/data/2014/acs5?get=NAME,{params}&".format(params=",".join(fields_of_interest))
    q += "for=block+group:*&in=state:{st}+county:{cy}".format(st=state,cy=county)
    q += "&key={key}".format(key='a6e317918af5d4097d792cabd992f41f2691b75e')
    try:
        response = urlopen(q)
    except:
        print 'NO DATA FOR: ', state, county
    resp_val = response.read().replace("null","None")
    if not len(resp_val):
        print 'NO DATA FOR: ', state, county
        return None
    kittens = eval(resp_val)
    headers = kittens.pop(0)
    df = pd.DataFrame(kittens,columns=headers)
    return df

bg_data = []
for i,state_county in enumerate(counties):
    #if i < len(bg_data):
    #    continue
    state, county = state_county.split("_")
    
    bg_data.append(get_loc_data(state,county))
    if i % 100 == 0:
        print i

2237 51476
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
NO DATA FOR:  51 515
1400
1500
1600
NO DATA FOR:  78 020
1700
1800
1900
2000
2100
2200


In [6]:
all_geo_data = pd.DataFrame()
for i, df in enumerate(bg_data):
    all_geo_data = all_geo_data.append(df,ignore_index=True)
    if i % 1000 == 0:
        print i
all_geo_data[fields_of_interest]=all_geo_data[fields_of_interest].astype('float')
all_geo_data.columns =  ['county_name','total_pop','male_pop','female_pop','white_pop',
                         'black_pop','med_age','n_home','med_income','not_hisp_pop','hisp_pop','gini',
                         'per_cap_inc','per_cap_white','per_cap_black','per_cap_hisp','tot_in_lab',
                         'tot_not_in_lab','state','county','tract','block_group']

0
1000
2000


In [10]:
user_data = []
c = io.open("../data/ferg_geo_data/uid_to_census_block.txt")
for line in c:
    uid, x = line.split()
    if uid not in gend_name:
        continue
    state, county, tract,block_group, block = x[:2], x[2:5],x[5:11],x[11],x[12:15]
    user_data.append([uid,state,county,tract,block_group])
c.close()
print len(user_data)
user_data = pd.DataFrame(user_data)
user_data.columns = ['uid','state','county','tract','block_group']

58148


In [21]:
user_geo_data = pd.merge(user_data,all_geo_data,
                         left_on=['state','county','tract','block_group'],
                         right_on=['state','county','tract','block_group'])

In [36]:
user_geo_data = user_geo_data[user_geo_data.total_pop > 0]

In [46]:
# construct and save out all user info
all_user_data = pd.merge(user_geo_data,gend_name, on='uid')
all_user_data.to_csv("../data/ferg_geo_data/all_user_geo_gender.csv")

In [23]:
# any exploratory plotting
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from math import log
ax = sns.jointplot(all_user_data.b_percent.apply(lambda x: log(x+1)),all_user_data.total_pop.apply(lambda x: log(x+1)))

In [81]:
# create cells for demographics, taken in from get_user_data.ipynb
import pandas as pd

all_user_data = pd.read_csv("../data/ferg_geo_data/all_user_geo_gender.csv")
# create cells for demographics
all_user_data['cell'] = 0
all_user_data.loc[(all_user_data.b_percent > all_user_data.b_percent.mean()) & (all_user_data.gender == 'M'),'cell'] = 0
all_user_data.loc[(all_user_data.b_percent > all_user_data.b_percent.mean()) & (all_user_data.gender == 'F'),'cell'] = 1
all_user_data.loc[(all_user_data.b_percent <= all_user_data.b_percent.mean()) & (all_user_data.gender == 'M'),'cell'] = 2
all_user_data.loc[(all_user_data.b_percent <= all_user_data.b_percent.mean()) & (all_user_data.gender == 'F'),'cell'] = 3
all_user_data.cell.value_counts()

cell_out = io.open("../data/ferg_geo_data/user_to_cell.tsv","w")
for k,v in zip(list(all_user_data.uid), list(all_user_data.cell)):
    cell_out.write(unicode(k) + "\t"+unicode(v) + "\n")
cell_out.close()